# Adaboost-Bagging(1)


In [ ]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report  # Import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

data = pd.read_csv('datawithTime.csv')

data['CARSHZD_categories'] = pd.cut(data['CARSHZD'], bins=[-float('inf'), 0, 2, float('inf')], labels=['0', '1 - 2', '>2'])

target_columns = ['CARSDMG', 'CASKLD', 'CARSHZD', 'EVACUATE', 'EQPDMG', 'CASINJ', 'ACCDMG', 'TRKDMG', 'CARSHZD_categories']
target_column = 'CARSHZD_categories'

feature_columns = [col for col in data.columns if col not in target_columns]

n_estimators = 150
estimator = RandomForestClassifier(n_estimators=100)  # You can adjust n_estimators as needed
learning_rate = 0.03

X_train, X_test, y_train, y_test = train_test_split(data[feature_columns], data[target_column], test_size=0.2, random_state=42)

test_set_predictions = []

for i in range(20):
    train_data, _ = train_test_split(X_train.join(y_train), train_size=0.3, random_state=i)
    X_train_sampled = train_data[feature_columns]
    y_train_sampled = train_data[target_column]

    ada = AdaBoostClassifier(base_estimator=estimator, n_estimators=n_estimators, learning_rate=learning_rate, random_state=i)
    ada.fit(X_train_sampled, y_train_sampled)

    test_set_predictions.append(ada.predict(X_test))

final_predictions = []

for i in range(len(y_test)):
    all_predictions = [predictions[i] for predictions in test_set_predictions]
    most_common_group = Counter(all_predictions).most_common(1)[0][0]
    final_predictions.append(most_common_group)

accuracy = accuracy_score(y_test, final_predictions)
print(f'Accuracy: {accuracy}')

final_confusion_matrix = confusion_matrix(y_test, final_predictions)

classification_rep = classification_report(y_test, final_predictions, target_names=['0', '1 - 2', '>2'])
print("Classification Report:\n", classification_rep)

plt.figure(figsize=(8, 6))
labels = ['0', '1 - 2', '>2']
sns.heatmap(final_confusion_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Ensemble Confusion Matrix (Voting Strategy)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, final_predictions)
print(f'Accuracy: {accuracy}')
